In [1]:
import pandas as pd
import folium
import requests
import geopy
import geopandas as gpd
from geopy.extra.rate_limiter import RateLimiter
from functools import partial
from geopy.geocoders import Nominatim
import json
from shapely.geometry import Point
import re 

from flatten_json import flatten
from folium.plugins import FastMarkerCluster

In [4]:
#single upload Census Retrieval

def census_retrieval(state):

    df_addresses_wo_dups = pd.read_csv(f'./data/addresses_wo_dups/{state}_addresses_wo_dups.csv').reset_index(drop=True)

    endpoint2 = 'https://geocoding.geo.census.gov/geocoder/geographies/address'
    df_addresses_wo_dups['output'] = ""
    for index, item in df_addresses_wo_dups.iterrows():
        print(index)
        params2 = {'benchmark' : 'Public_AR_Current',
                   'vintage' : 'Census2010_Current',
                   'street' : str(df_addresses_wo_dups.loc[index, 'address_1']),
                   'city' : str(df_addresses_wo_dups.loc[index, 'city']),
                     'state' : str(df_addresses_wo_dups.loc[index, 'state']),
                       'format': 'json',
                  'layers' : 'all'}

        df_addresses_wo_dups.loc[index, 'output'] = str(requests.get(endpoint2, params2).json())
        print(df_addresses_wo_dups.loc[index, 'output'])
    df_addresses_wo_dups.to_csv(f'./data/addresses_wo_dups_census/{state}_addresses_wo_dups_census.csv')

In [5]:
census_retrieval('KY')

0
{'result': {'input': {'benchmark': {'id': '4', 'benchmarkName': 'Public_AR_Current', 'benchmarkDescription': 'Public Address Ranges - Current Benchmark', 'isDefault': False}, 'vintage': {'id': '410', 'vintageName': 'Census2010_Current', 'vintageDescription': 'Census2010 Vintage - Current Benchmark', 'isDefault': False}, 'address': {'street': '1000 E JOHN ROWAN BLVD STE 101', 'city': 'BARDSTOWN', 'state': 'KY'}}, 'addressMatches': [{'matchedAddress': '1000 E JOHN ROWAN BLVD, BARDSTOWN, KY, 40004', 'coordinates': {'x': -85.45752, 'y': 37.8254}, 'tigerLine': {'tigerLineId': '35151234', 'side': 'R'}, 'addressComponents': {'fromAddress': '1352', 'toAddress': '500', 'preQualifier': '', 'preDirection': 'E', 'preType': '', 'streetName': 'JOHN ROWAN', 'suffixType': 'BLVD', 'suffixDirection': '', 'suffixQualifier': '', 'city': 'BARDSTOWN', 'state': 'KY', 'zip': '40004'}, 'geographies': {'Public Use Microdata Areas': [{'POP100': 106913, 'GEOID': '2101200', 'CENTLAT': '+37.6663898', 'AREAWATER':

{'result': {'input': {'benchmark': {'id': '4', 'benchmarkName': 'Public_AR_Current', 'benchmarkDescription': 'Public Address Ranges - Current Benchmark', 'isDefault': False}, 'vintage': {'id': '410', 'vintageName': 'Census2010_Current', 'vintageDescription': 'Census2010 Vintage - Current Benchmark', 'isDefault': False}, 'address': {'street': '798 PORTLAND AVE', 'city': 'BARDSTOWN', 'state': 'KY'}}, 'addressMatches': [{'matchedAddress': '798 PORTLAND AVE, BARDSTOWN, KY, 40004', 'coordinates': {'x': -85.46927, 'y': 37.824345}, 'tigerLine': {'tigerLineId': '35147911', 'side': 'L'}, 'addressComponents': {'fromAddress': '700', 'toAddress': '798', 'preQualifier': '', 'preDirection': '', 'preType': '', 'streetName': 'PORTLAND', 'suffixType': 'AVE', 'suffixDirection': '', 'suffixQualifier': '', 'city': 'BARDSTOWN', 'state': 'KY', 'zip': '40004'}, 'geographies': {'Public Use Microdata Areas': [{'POP100': 106913, 'GEOID': '2101200', 'CENTLAT': '+37.6663898', 'AREAWATER': 49596227, 'BASENAME': 'L

KeyboardInterrupt: 

In [6]:
df_addresses_wo_dups

NameError: name 'df_addresses_wo_dups' is not defined

In [32]:
def expand(output):
        df = pd.json_normalize(pd.json_normalize(output)['result.addressMatches'][0]).transpose()
        return df

In [53]:
output = {'result': {'input': {'benchmark': {'id': '4', 'benchmarkName': 'Public_AR_Current', 'benchmarkDescription': 'Public Address Ranges - Current Benchmark', 'isDefault': False}, 'vintage': {'id': '410', 'vintageName': 'Census2010_Current', 'vintageDescription': 'Census2010 Vintage - Current Benchmark', 'isDefault': False}, 'address': {'street': '798 PORTLAND AVE', 'city': 'BARDSTOWN', 'state': 'KY'}}, 'addressMatches': [{'matchedAddress': '798 PORTLAND AVE, BARDSTOWN, KY, 40004', 'coordinates': {'x': -85.46927, 'y': 37.824345}, 'tigerLine': {'tigerLineId': '35147911', 'side': 'L'}, 'addressComponents': {'fromAddress': '700', 'toAddress': '798', 'preQualifier': '', 'preDirection': '', 'preType': '', 'streetName': 'PORTLAND', 'suffixType': 'AVE', 'suffixDirection': '', 'suffixQualifier': '', 'city': 'BARDSTOWN', 'state': 'KY', 'zip': '40004'}, 'geographies': {'Public Use Microdata Areas': [{'POP100': 106913, 'GEOID': '2101200', 'CENTLAT': '+37.6663898', 'AREAWATER': 49596227, 'BASENAME': 'Lincoln Trail Area Development District (East)', 'STATE': '21', 'OID': '217404485564966', 'LSADC': 'P0', 'FUNCSTAT': 'S', 'INTPTLAT': '+37.6591545', 'NAME': 'Lincoln Trail Area Development District (East) PUMA', 'OBJECTID': 1608, 'CENTLON': '-085.4606507', 'AREALAND': 3938297343, 'INTPTLON': '-085.4600145', 'HU100': 44459, 'MTFCC': 'G6120', 'PUMA': '01200'}], 'Traffic Analysis Districts': [{'POP100': 23681, 'GEOID': '2100000000000097', 'CENTLAT': '+37.8615160', 'AREAWATER': 6595195, 'BASENAME': '00000097', 'OID': '225404471187661', 'LSADC': 'TD', 'MPOCE': '21000000', 'FUNCSTAT': 'S', 'INTPTLAT': '+37.8689244', 'NAME': 'TAD 00000097', 'OBJECTID': 2932, 'CENTLON': '-085.5024994', 'TAD': '00000097', 'AREALAND': 490350426, 'INTPTLON': '-085.4976765', 'HU100': 9824, 'MTFCC': 'G6300'}], 'ZIP Code Tabulation Areas': [{'POP100': 28245, 'GEOID': '40004', 'CENTLAT': '+37.8045079', 'AREAWATER': 5379248, 'BASENAME': '40004', 'OID': '221404258278789', 'LSADC': 'Z5', 'ZCTA5': '40004', 'FUNCSTAT': 'S', 'INTPTLAT': '+37.8031198', 'NAME': 'ZCTA5 40004', 'OBJECTID': 16505, 'CENTLON': '-085.4649723', 'AREALAND': 374022721, 'INTPTLON': '-085.4659312', 'HU100': 11791, 'MTFCC': 'G6350', 'ZCTA5CC': 'B5'}], 'Urban Clusters': [{'POP100': 18702, 'GEOID': '05059', 'CENTLAT': '+37.8334585', 'AREAWATER': 382446, 'BASENAME': 'Bardstown, KY', 'OID': '27040121231245', 'UA': '05059', 'LSADC': '76', 'FUNCSTAT': 'S', 'INTPTLAT': '+37.8357623', 'NAME': 'Bardstown, KY Urban Cluster', 'OBJECTID': 3104, 'CENTLON': '-085.4616325', 'AREALAND': 44071163, 'INTPTLON': '-085.4659997', 'HU100': 7927, 'MTFCC': 'G3500'}], 'Counties': [{'POP100': 43437, 'GEOID': '21179', 'CENTLAT': '+37.8050571', 'AREAWATER': 16973349, 'STATE': '21', 'BASENAME': 'Nelson', 'OID': '27540121230222', 'LSADC': '06', 'FUNCSTAT': 'A', 'INTPTLAT': '+37.8031198', 'NAME': 'Nelson County', 'OBJECTID': 580, 'CENTLON': '-085.4661726', 'COUNTYCC': 'H1', 'COUNTYNS': '00516936', 'AREALAND': 1081379029, 'INTPTLON': '-085.4659312', 'HU100': 18075, 'MTFCC': 'G4020', 'UR': 'M', 'COUNTY': '179'}], 'Unified School Districts': [{'POP100': 12939, 'LOGRADE': 'PK', 'GEOID': '2100270', 'CENTLAT': '+37.8214588', 'AREAWATER': 437736, 'STATE': '21', 'BASENAME': 'Bardstown Independent School District', 'OID': '28440121231432', 'SDTYP': '', 'LSADC': '00', 'FUNCSTAT': 'E', 'INTPTLAT': '+37.8214766', 'SDUNI': '00270', 'NAME': 'Bardstown Independent School District', 'OBJECTID': 5250, 'CENTLON': '-085.4700708', 'HIGRADE': '12', 'AREALAND': 36489668, 'INTPTLON': '-085.4707839', 'HU100': 5697, 'MTFCC': 'G5420', 'UR': 'M'}], '2012 State Legislative Districts - Lower': [{'POP100': '', 'GEOID': '21050', 'CENTLAT': '+37.8233363', 'SLDL': '050', 'AREAWATER': 17772115, 'STATE': '21', 'BASENAME': '50', 'OID': '213404699440677', 'LSADC': 'LL', 'FUNCSTAT': 'N', 'INTPTLAT': '+37.8246134', 'NAME': 'State House District 50', 'OBJECTID': 2602, 'CENTLON': '-085.4650034', 'LSY': '2012', 'AREALAND': 1186112777, 'INTPTLON': '-085.4652257', 'HU100': '', 'MTFCC': 'G5220', 'LDTYP': 'O'}], '2012 State Legislative Districts - Upper': [{'POP100': '', 'GEOID': '21014', 'CENTLAT': '+37.6659953', 'AREAWATER': 75853790, 'STATE': '21', 'BASENAME': '14', 'OID': '212404699442556', 'SLDU': '014', 'LSADC': 'LU', 'FUNCSTAT': 'N', 'INTPTLAT': '+37.6751070', 'NAME': 'State Senate District 14', 'OBJECTID': 81, 'CENTLON': '-085.2514650', 'LSY': '2012', 'AREALAND': 4073983870, 'INTPTLON': '-085.2610704', 'HU100': '', 'MTFCC': 'G5210', 'LDTYP': 'O'}], 'States': [{'STATENS': '01779786', 'POP100': 4339367, 'GEOID': '21', 'CENTLAT': '+37.5351707', 'AREAWATER': 2367706664, 'STATE': '21', 'BASENAME': 'Kentucky', 'STUSAB': 'KY', 'OID': '27440100360204', 'LSADC': '00', 'FUNCSTAT': 'A', 'INTPTLAT': '+37.5336844', 'DIVISION': '6', 'NAME': 'Kentucky', 'REGION': '3', 'OBJECTID': 47, 'CENTLON': '-085.3018121', 'AREALAND': 102283339482, 'INTPTLON': '-085.2929801', 'HU100': 1927164, 'MTFCC': 'G4000', 'UR': 'M'}], 'Census Block Groups': [{'POP100': 2137, 'GEOID': '211799303012', 'CENTLAT': '+37.8211467', 'AREAWATER': 8910, 'STATE': '21', 'BASENAME': '2', 'OID': '208403716701997', 'LSADC': 'BG', 'FUNCSTAT': 'S', 'INTPTLAT': '+37.8184744', 'NAME': 'Block Group 2', 'OBJECTID': 140842, 'TRACT': '930301', 'CENTLON': '-085.4767817', 'BLKGRP': '2', 'AREALAND': 4763021, 'INTPTLON': '-085.4771446', 'HU100': 1001, 'MTFCC': 'G5030', 'UR': 'M', 'COUNTY': '179'}], 'Combined Statistical Areas': [{'POP100': 1427483, 'GEOID': '350', 'CENTLAT': '+38.1558508', 'AREAWATER': 247385871, 'BASENAME': 'Louisville/Jefferson County--Elizabethtown--Scottsburg, KY-IN', 'OID': '26140104230199', 'LSADC': 'M0', 'FUNCSTAT': 'S', 'INTPTLAT': '+38.1548655', 'NAME': 'Louisville/Jefferson County--Elizabethtown--Scottsburg, KY-IN CSA', 'OBJECTID': 31, 'CSA': '350', 'CENTLON': '-085.7242514', 'INTPTLON': '-085.7251652', 'AREALAND': 13434168118, 'HU100': 619710, 'MTFCC': 'G3100'}], 'County Subdivisions': [{'COUSUB': '90152', 'POP100': 26012, 'GEOID': '2117990152', 'CENTLAT': '+37.7964087', 'AREAWATER': 4595687, 'STATE': '21', 'BASENAME': 'Bardstown', 'OID': '27640121230490', 'LSADC': '22', 'FUNCSTAT': 'S', 'INTPTLAT': '+37.7942333', 'NAME': 'Bardstown CCD', 'OBJECTID': 32818, 'CENTLON': '-085.4475978', 'COUSUBCC': 'Z5', 'AREALAND': 316032373, 'INTPTLON': '-085.4493253', 'HU100': 10905, 'MTFCC': 'G4040', 'COUSUBNS': '01936864', 'UR': 'M', 'COUNTY': '179'}], 'Census Divisions': [{'NATIONFP': 'US', 'POP100': 18432505, 'GEOID': '6', 'CENTLAT': '+34.5139527', 'ALANDHIST': '', 'AREAWATER': 13234282778, 'EFFDATE': '', 'BASENAME': 'East South Central', 'OID': '2734020719813', 'LSADC': '69', 'FUNCSTAT': 'S', 'INTPTLAT': '+34.5174373', 'DIVISION': '6', 'NAME': 'East South Central Division', 'REGION': '3', 'OBJECTID': 3, 'CENTLON': '-087.1639714', 'AREALAND': 461791114828, 'INTPTLON': '-087.1675441', 'HU100': 8185869, 'VINTAGE': '40', 'AWATERHIST': '', 'MTFCC': 'G1200', 'ESTABDATE': ''}], '113th Congressional Districts': [{'POP100': '', 'GEOID': '2102', 'CENTLAT': '+37.5365197', 'CDSESSN': '113', 'AREAWATER': 388119174, 'STATE': '21', 'BASENAME': '2', 'OID': '211404487269774', 'LSADC': 'C2', 'FUNCSTAT': 'N', 'INTPTLAT': '+37.5364651', 'NAME': 'Congressional District 2', 'OBJECTID': 105, 'CENTLON': '-085.9897105', 'AREALAND': 18601341638, 'INTPTLON': '-085.9932321', 'HU100': '', 'MTFCC': 'G5200', 'CD113': '02'}], 'Traffic Analysis Zones': [{'POP100': 2007, 'GEOID': '2117900309025', 'CENTLAT': '+37.8204084', 'AREAWATER': 8092, 'STATE': '21', 'BASENAME': '00309025', 'OID': '219404471186733', 'LSADC': 'TZ', 'MPOCE': '21000000', 'TAZ': '00309025', 'FUNCSTAT': 'S', 'INTPTLAT': '+37.8184744', 'NAME': 'TAZ 00309025', 'OBJECTID': 158832, 'CENTLON': '-085.4763930', 'TAD': '00000097', 'AREALAND': 4349813, 'INTPTLON': '-085.4771446', 'HU100': 907, 'MTFCC': 'G6320', 'COUNTY': '179'}], 'Census Regions': [{'POP100': 114555744, 'GEOID': '3', 'CENTLAT': '+33.5458123', 'AREAWATER': 148742458313, 'BASENAME': 'South', 'OID': '2724020719822', 'LSADC': '68', 'FUNCSTAT': 'S', 'INTPTLAT': '+33.5528399', 'REGION': '3', 'NAME': 'South Region', 'OBJECTID': 1, 'CENTLON': '-090.1661028', 'AREALAND': 2249835399274, 'INTPTLON': '-090.1649735', 'HU100': 49980829, 'MTFCC': 'G1100'}], 'Metropolitan Statistical Areas': [{'POP100': 1283566, 'GEOID': '31140', 'CENTLAT': '+38.2386116', 'AREAWATER': 218490863, 'BASENAME': 'Louisville/Jefferson County, KY-IN', 'OID': '26240104230198', 'LSADC': 'M1', 'FUNCSTAT': 'S', 'INTPTLAT': '+38.2395982', 'NAME': 'Louisville/Jefferson County, KY-IN Metro Area', 'CSA': '350', 'OBJECTID': 139, 'CENTLON': '-085.6886723', 'CBSA': '31140', 'AREALAND': 10649021780, 'INTPTLON': '-085.6875001', 'HU100': 559837, 'MTFCC': 'G3110'}], '111th Congressional Districts': [{'POP100': 760032, 'GEOID': '2102', 'CENTLAT': '+37.5737319', 'CDSESSN': '111', 'AREAWATER': 411754385, 'STATE': '21', 'BASENAME': '2', 'OID': '21140106677610', 'LSADC': 'C2', 'FUNCSTAT': 'N', 'INTPTLAT': '+37.5832540', 'NAME': 'Congressional District 2', 'OBJECTID': 240, 'CENTLON': '-086.0019140', 'AREALAND': 19449284040, 'INTPTLON': '-085.9948687', 'HU100': 324758, 'MTFCC': 'G5200', 'CD111': '02'}], 'Census Tracts': [{'POP100': 5112, 'GEOID': '21179930301', 'CENTLAT': '+37.8351792', 'AREAWATER': 900853, 'STATE': '21', 'BASENAME': '9303.01', 'OID': '207403716690770', 'LSADC': 'CT', 'FUNCSTAT': 'S', 'INTPTLAT': '+37.8328050', 'NAME': 'Census Tract 9303.01', 'OBJECTID': 63894, 'TRACT': '930301', 'CENTLON': '-085.5248304', 'AREALAND': 55465227, 'INTPTLON': '-085.5579297', 'HU100': 2183, 'MTFCC': 'G5020', 'UR': 'M', 'COUNTY': '179'}], 'Census Blocks': [{'GEOID': '211799303012034', 'STATE': '21', 'BASENAME': '2034', 'LSADC': 'BK', 'INTPTLAT': '+37.8259181', 'OBJECTID': 7423908, 'BLKGRP': '2', 'AREALAND': 80725, 'HU100': 0, 'VINTAGE': '40', 'LWBLKTYP': 'L', 'UR': 'U', 'COUNTY': '179', 'SUFFIX': '', 'TABBLKSUFX2': '', 'POP100': 0, 'CENTLAT': '+37.8259181', 'BLOCK': '2034', 'AREAWATER': 0, 'OID': '210404018974341', 'FUNCSTAT': 'S', 'NAME': 'Block 2034', 'TRACT': '930301', 'CENTLON': '-085.4709375', 'ACT': 'B3', 'INTPTLON': '-085.4709375', 'MTFCC': 'G5040'}], '2010 State Legislative Districts - Upper': [{'POP100': 120817, 'GEOID': '21014', 'CENTLAT': '+37.6659953', 'AREAWATER': 75853790, 'STATE': '21', 'BASENAME': '14', 'OID': '21240115120179', 'LSADC': 'LU', 'SLDU': '014', 'FUNCSTAT': 'N', 'INTPTLAT': '+37.6751070', 'NAME': 'State Senate District 14', 'OBJECTID': 865, 'CENTLON': '-085.2514650', 'LSY': '2010', 'AREALAND': 4073983870, 'INTPTLON': '-085.2610704', 'HU100': 52106, 'MTFCC': 'G5210', 'LDTYP': 'O'}], '2010 State Legislative Districts - Lower': [{'POP100': 49478, 'GEOID': '21050', 'CENTLAT': '+37.8233363', 'SLDL': '050', 'AREAWATER': 17772115, 'STATE': '21', 'BASENAME': '50', 'OID': '21340115962188', 'LSADC': 'LL', 'FUNCSTAT': 'N', 'INTPTLAT': '+37.8246134', 'NAME': 'State House District 50', 'OBJECTID': 1422, 'CENTLON': '-085.4650034', 'LSY': '2010', 'AREALAND': 1186112777, 'INTPTLON': '-085.4652257', 'HU100': 20437, 'MTFCC': 'G5220', 'LDTYP': 'O'}]}}]}}
expand(output).loc['geographies.Metropolitan Statistical Areas'][0]

[{'POP100': 1283566,
  'GEOID': '31140',
  'CENTLAT': '+38.2386116',
  'AREAWATER': 218490863,
  'BASENAME': 'Louisville/Jefferson County, KY-IN',
  'OID': '26240104230198',
  'LSADC': 'M1',
  'FUNCSTAT': 'S',
  'INTPTLAT': '+38.2395982',
  'NAME': 'Louisville/Jefferson County, KY-IN Metro Area',
  'CSA': '350',
  'OBJECTID': 139,
  'CENTLON': '-085.6886723',
  'CBSA': '31140',
  'AREALAND': 10649021780,
  'INTPTLON': '-085.6875001',
  'HU100': 559837,
  'MTFCC': 'G3110'}]

In [40]:
expand(output)

,0
matchedAddress,"798 PORTLAND AVE, BARDSTOWN, KY, 40004"
coordinates.x,-85.46927
coordinates.y,37.824345
tigerLine.tigerLineId,35147911
tigerLine.side,L
addressComponents.fromAddress,700
addressComponents.toAddress,798
addressComponents.preQualifier,
addressComponents.preDirection,
addressComponents.preType,
